In [1]:
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import DBSCAN

from gower.gower_dist import *

# Simple example from D'Orazio

In [2]:
df = pd.DataFrame({"Sex1": ["M", "M", "F", "F", "F", "M", "M", "F", "F", "F"],
                   "Sex2": ["M", "M", "F", "F", "F", "F", "F", "M", "M", "M"],
                   "Age1": [15] * 10,
                   "Age2": [15, 36, 58, 78, 100, 15, 36, 58, 78, 100]})
df

,Sex1,Sex2,Age1,Age2
0,M,M,15,15
1,M,M,15,36
2,F,F,15,58
3,F,F,15,78
4,F,F,15,100
5,M,F,15,15
6,M,F,15,36
7,F,M,15,58
8,F,M,15,78
9,F,M,15,100


In [3]:
distances = np.diag(gower_matrix(df[["Sex1", "Age1"]].to_numpy(), df[["Sex2", "Age2"]].to_numpy(), weight="uniform"))
print(distances)
df.loc[np.argsort(distances)]

  0%|          | 0/10 [00:00<?, ?it/s]

[0.         0.12352941 0.25294118 0.37058824 0.5        0.5
 0.62352941 0.75294118 0.87058824 1.        ]


,Sex1,Sex2,Age1,Age2
0,M,M,15,15
1,M,M,15,36
2,F,F,15,58
3,F,F,15,78
4,F,F,15,100
5,M,F,15,15
6,M,F,15,36
7,F,M,15,58
8,F,M,15,78
9,F,M,15,100


In [4]:
distances = np.diag(gower_matrix(df[["Sex1", "Age1"]].to_numpy(), df[["Sex2", "Age2"]].to_numpy(), R=(25, 75)))
print(distances)
df.loc[np.argsort(distances)]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

[0.         0.36257249 0.74241035 0.74241035 0.74241035 0.25758965
 0.62016215 1.         1.         1.        ]


,Sex1,Sex2,Age1,Age2
0,M,M,15,15
5,M,F,15,15
1,M,M,15,36
6,M,F,15,36
2,F,F,15,58
3,F,F,15,78
4,F,F,15,100
7,F,M,15,58
8,F,M,15,78
9,F,M,15,100


# Some plots

In [5]:
colors = mcolors._colors_full_map
MAX = 40

# Dog adoption dataset

In [6]:
dd = pd.read_csv("dog_adoption/allDogDescriptions.csv", index_col=0)
dd.set_index("id", inplace=True, drop=True)


# get rid of columns with the most destructive nans

def isolate_nans(column):
    all_other_nans = nans[[c for c in cols if c != column]].sum(axis=1)
    mask = all_other_nans < np.median(all_other_nans)
    mask &= nans[column]
    return mask.sum()


nans = dd.isna()
cols = dd.columns
isolated_nans = {c: isolate_nans(c) for c in cols}
median_count = np.median([v for v in isolated_nans.values() if v > 0])
for column, count in isolated_nans.items():
    if count > median_count:
        print(column)
        dd.drop(column, axis=1, inplace=True)
dd.dropna(inplace=True)  # dd.mode().iloc[0]

# make bools strings

is_bool = dd.dtypes == bool
dd.loc[:, is_bool] = dd[dd.columns[is_bool]].astype(str)

dd.dtypes

breed_secondary
color_primary
color_secondary
color_tertiary
declawed
env_cats


org_id             object
url                object
type.x             object
species            object
breed_primary      object
breed_mixed        object
breed_unknown      object
age                object
sex                object
size               object
coat               object
fixed              object
house_trained      object
special_needs      object
shots_current      object
env_children       object
env_dogs           object
name               object
status             object
posted             object
contact_city       object
contact_state      object
contact_zip        object
contact_country    object
stateQ             object
accessed           object
type.y             object
description        object
dtype: object

We can also use the categorical weight function to measure the performance
of clustering algorithms and thereby tune the parameters of DBSCAN.

In [7]:
matrix = gower_matrix(dd.to_numpy(), chunksize=20)

  0%|          | 0/28 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/13729 [00:00<?, ?it/s]

In [8]:
samples = [{"eps": z / 10, "min_samples": a} for z in range(1, 11) for a in range(1, 11)]
results = process_map(partial(do_it, matrix=matrix), samples, chunksize=1)
best_params = max(results, key=lambda z: z[1])
best_params

  0%|          | 0/100 [00:00<?, ?it/s]

({'eps': 0.5, 'min_samples': 1}, 0.9078390917547138)

In [9]:
results

[({'eps': 0.1, 'min_samples': 1}, 0.0),
 ({'eps': 0.1, 'min_samples': 2}, 0),
 ({'eps': 0.1, 'min_samples': 3}, 0),
 ({'eps': 0.1, 'min_samples': 4}, 0),
 ({'eps': 0.1, 'min_samples': 5}, 0),
 ({'eps': 0.1, 'min_samples': 6}, 0),
 ({'eps': 0.1, 'min_samples': 7}, 0),
 ({'eps': 0.1, 'min_samples': 8}, 0),
 ({'eps': 0.1, 'min_samples': 9}, 0),
 ({'eps': 0.1, 'min_samples': 10}, 0),
 ({'eps': 0.2, 'min_samples': 1}, 0.0009104814625974234),
 ({'eps': 0.2, 'min_samples': 2}, 0.0009104814625974234),
 ({'eps': 0.2, 'min_samples': 3}, 0),
 ({'eps': 0.2, 'min_samples': 4}, 0),
 ({'eps': 0.2, 'min_samples': 5}, 0),
 ({'eps': 0.2, 'min_samples': 6}, 0),
 ({'eps': 0.2, 'min_samples': 7}, 0),
 ({'eps': 0.2, 'min_samples': 8}, 0),
 ({'eps': 0.2, 'min_samples': 9}, 0),
 ({'eps': 0.2, 'min_samples': 10}, 0),
 ({'eps': 0.3, 'min_samples': 1}, 0.21714837797314568),
 ({'eps': 0.3, 'min_samples': 2}, 0.21714837797314568),
 ({'eps': 0.3, 'min_samples': 3}, 0.13159953974749194),
 ({'eps': 0.3, 'min_samples'

In [10]:
dd["gower_dbscan"] = DBSCAN(metric="precomputed", **best_params[0]).fit_predict(matrix)
print(get_cat_weight(dd["gower_dbscan"]))
np.unique(dd["gower_dbscan"], return_counts=True)

0.9078390917547138


(array([   0,    1,    2, ..., 2034, 2035, 2036]),
 array([ 19, 101,   2, ...,   5,   1,   1]))

In [11]:
dd["label_count"] = dd.groupby("gower_dbscan").transform("count").iloc[:, 0]
dd.sort_values(["label_count", "gower_dbscan"], ascending=[False, True], inplace=True)
dd

,org_id,url,type.x,species,breed_primary,breed_mixed,breed_unknown,age,sex,size,...,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,gower_dbscan,label_count
id,,,,,,,,,,,,,,,,,,,,,
46021535,TX2394,https://www.petfinder.com/dog/brody-bear-46021...,Dog,Dog,Schnauzer,True,False,Baby,Male,Small,...,Brattleboro,VT,05301,US,CT,2019-09-20,Dog,Howdy! Caught you looking! My name is Brody! ...,234,250
46021378,TX2394,https://www.petfinder.com/dog/brody-46021378/c...,Dog,Dog,Schnauzer,True,False,Baby,Male,Small,...,Groton,CT,06340,US,CT,2019-09-20,Dog,Howdy! Caught you looking! My name is Brody! ...,234,250
46021361,TX2394,https://www.petfinder.com/dog/daisey-may-46021...,Dog,Dog,Weimaraner,True,False,Young,Female,Medium,...,Brattleboro,VT,05301,US,CT,2019-09-20,Dog,Howdy! Caught you looking! My name is Daisey M...,234,250
46021331,TX2394,https://www.petfinder.com/dog/daisey-mae-46021...,Dog,Dog,Weimaraner,True,False,Young,Female,Medium,...,Boston,MA,02128,US,CT,2019-09-20,Dog,Howdy! Caught you looking! My name is Daisey M...,234,250
46021276,TX2394,https://www.petfinder.com/dog/daisey-mae-3-460...,Dog,Dog,Weimaraner,True,False,Young,Female,Medium,...,Groton,CT,06340,US,CT,2019-09-20,Dog,Howdy! Caught you looking! My name is Daisey M...,234,250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45070846,WV80,https://www.petfinder.com/dog/manie-45070846/w...,Dog,Dog,Terrier,True,False,Adult,Male,Small,...,Scott Depot,WV,25560,US,WV,2019-09-20,Dog,Manie's birthday is 6/21/2018. He likes attent...,2030,1
44531731,WV04,https://www.petfinder.com/dog/coco-44531731/wv...,Dog,Dog,Pit Bull Terrier,True,False,Adult,Female,Medium,...,Fairmont,WV,26554,US,WV,2019-09-20,Dog,If you'd like to learn more about one of our a...,2031,1
44175060,WV37,https://www.petfinder.com/dog/skylee-44175060/...,Dog,Dog,Pug,True,False,Senior,Female,Medium,...,Dellslow,WV,26531,US,WV,2019-09-20,Dog,"Hi Skylee !! ??\nSkylee is a senior girl, who ...",2032,1


# Pokemon dataset

In [12]:
pokemon = pd.read_csv("pokemon/pokemon_alopez247.csv")

for c in pokemon.columns[pokemon.dtypes != object]:
    pokemon.loc[pokemon[c].isna(), c] = np.nanmedian(pokemon[c])
for c in pokemon.columns[pokemon.dtypes == object]:
    pokemon.loc[pokemon[c].isna(), c] = "none"

matrix2 = gower_matrix(pokemon.to_numpy(), knn=True, chunksize=20)
samples = [{"eps": 0.1 * z / 1000, "min_samples": 1} for z in range(1, 1001)]
results2 = process_map(partial(do_it, matrix=matrix2), samples, chunksize=20)
best_params2 = max(results2, key=lambda z: z[1])
print(best_params2)
pokemon["gower_dbscan"] = DBSCAN(metric="precomputed", **best_params2[0]).fit_predict(matrix2)
_, counts2 = np.unique(pokemon["gower_dbscan"], return_counts=True)
np.unique(counts2, return_counts=True)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/721 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

({'eps': 0.042, 'min_samples': 1}, 0.6641867713018119)


(array([ 1,  2,  3,  4,  7,  8, 15, 16, 17, 18, 37, 48, 72, 77, 80, 87]),
 array([144,  16,   8,   2,   3,   1,   1,   1,   2,   1,   1,   1,   1,
          1,   1,   1]))

In [13]:
pokemon["label_count"] = pokemon.groupby("gower_dbscan").transform("count").iloc[:, 0]
pokemon.sort_values(["label_count", "gower_dbscan"], ascending=[False, True], inplace=True)
pokemon


,Number,Name,Type_1,Type_2,Total,HP,Attack,Defense,Sp_Atk,Sp_Def,...,Pr_Male,Egg_Group_1,Egg_Group_2,hasMegaEvolution,Height_m,Weight_kg,Catch_Rate,Body_Style,gower_dbscan,label_count
494,495,Snivy,Grass,none,308,45,45,55,45,55,...,0.875,Field,Grass,False,0.61,8.1,45,bipedal_tailed,135,87
497,498,Tepig,Fire,none,308,65,63,45,45,45,...,0.875,Field,none,False,0.51,9.9,45,quadruped,135,87
500,501,Oshawott,Water,none,308,55,55,45,63,45,...,0.875,Field,none,False,0.51,5.9,45,bipedal_tailed,135,87
503,504,Patrat,Normal,none,255,45,55,39,35,39,...,0.500,Field,none,False,0.51,11.6,255,quadruped,135,87
505,506,Lillipup,Normal,none,275,45,60,45,25,45,...,0.500,Field,none,False,0.41,4.1,255,quadruped,135,87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,713,Avalugg,Ice,none,514,95,117,184,44,46,...,0.500,Monster,none,False,2.01,505.0,55,quadruped,179,1
713,714,Noibat,Flying,Dragon,245,40,30,35,45,40,...,0.500,Flying,none,False,0.51,8.0,190,two_wings,180,1
714,715,Noivern,Flying,Dragon,535,85,70,80,97,80,...,0.500,Flying,none,False,1.50,85.0,45,two_wings,181,1
717,718,Zygarde,Dragon,Ground,600,108,100,121,81,95,...,0.500,Undiscovered,none,False,5.00,305.0,3,serpentine_body,183,1


In [14]:
get_cat_weight(dd["gower_dbscan"])


0.9078390917547138

In [15]:
get_cat_weight(pokemon["gower_dbscan"])


0.6641867713018119

In [27]:
laptops = pd.read_csv("laptops/laptops_train.csv", index_col=0)
print(laptops.shape)
print(laptops.apply(lambda col: col.unique(), axis=0).apply(len))
laptops.drop("Operating System Version", axis=1, inplace=True)

matrix3 = gower_matrix(laptops.to_numpy(), R=(25, 75), chunksize=20)  # worse without R
samples = [{"eps": z / 1000, "min_samples": 1} for z in range(1, 1001)]
results3 = process_map(partial(do_it, matrix=matrix3), samples, chunksize=20)
best_params3 = max(results3, key=lambda z: z[1])
print(best_params3)
laptops["gower_dbscan"] = DBSCAN(metric="precomputed", **best_params3[0]).fit_predict(matrix3)
_, counts3 = np.unique(laptops["gower_dbscan"], return_counts=True)
np.unique(counts3, return_counts=True)

(977, 12)
Model Name                  488
Category                      6
Screen Size                  18
Screen                       38
CPU                         106
RAM                           8
 Storage                     36
GPU                          98
Operating System              7
Operating System Version      5
Weight                      166
Price                       639
dtype: int64


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/977 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

({'eps': 0.068, 'min_samples': 1}, 0.5416400184785664)


(array([  1,   2,   3,   4,   5,   6,   8,  21,  30,  61,  76, 258, 301]),
 array([136,  17,   5,   3,   1,   2,   2,   1,   1,   1,   1,   1,   1]))

In [28]:
laptops["label_count"] = laptops.groupby("gower_dbscan").transform("count").iloc[:, 0]
laptops.sort_values(["label_count", "gower_dbscan"], ascending=[False, True], inplace=True)
laptops


,Model Name,Category,Screen Size,Screen,CPU,RAM,Storage,GPU,Operating System,Weight,Price,gower_dbscan,label_count
Manufacturer,,,,,,,,,,,,,
HP,250 G6,Notebook,"15.6""",Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,5112900.00,2,301
Acer,Aspire 3,Notebook,"15.6""",1366x768,AMD A9-Series 9420 3GHz,4GB,500GB HDD,AMD Radeon R5,Windows,2.1kg,3556800.00,2,301
HP,250 G6,Notebook,"15.6""",1366x768,Intel Core i5 7200U 2.5GHz,4GB,500GB HDD,Intel HD Graphics 620,No OS,1.86kg,3502558.80,2,301
HP,250 G6,Notebook,"15.6""",Full HD 1920x1080,Intel Core i3 6006U 2GHz,4GB,500GB HDD,Intel HD Graphics 520,No OS,1.86kg,3067651.08,2,301
Dell,Inspiron 3567,Notebook,"15.6""",Full HD 1920x1080,Intel Core i3 6006U 2GHz,4GB,256GB SSD,AMD Radeon R5 M430,Windows,2.2kg,4436218.80,2,301
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Asus,Chromebook Flip,2 in 1 Convertible,"12.5""",Full HD 1920x1080,Intel Core M M7-6Y75 1.2GHz,8GB,64GB Flash Storage,Intel HD Graphics 515,Chrome OS,1.2kg,10305828.00,167,1
HP,ProBook x360,2 in 1 Convertible,"11.6""",Touchscreen 1366x768,Intel Pentium Quad Core N4200 1.1GHz,4GB,256GB SSD,Intel HD Graphics 505,Windows,1.45kg,6891300.00,168,1
Dell,XPS 13,Ultrabook,"13.3""",Quad HD+ 3200x1800,Intel Core i7 6500U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows,1.3kg,11275056.00,169,1
